In [1]:
using JuMP
using HiGHS

## 1) Formulate a DRO model for the newsvendor problem. Assume a known expected demand of 100 and a support set of [50,150] to build the ambiguity set. Develop and solve the single level problem.

$$
\begin{aligned}
    &\min && cx + max_{q \geq 0} \Sigma_s q_s Q(x,d_s) \\
    & st && 0 \leq x \leq u \\
        &&& \Sigma_s q_s = 1\\
        &&& \Sigma_s q_sd_s = \bar{d}\\
        &&& Q(x,d) =
     \begin{aligned}[t]
        &\min && -qy-rz \\
            & st && y \leq d \\
            &&& y+z \leq x\\
            &&& y, z \geq 0\\
     \end{aligned}\\
\end{aligned}
$$

$$
\begin{aligned}
    &\min && cx + D(x,d) \\
    & st && 0 \leq x \leq u \\
        &&& D(x,d) =
     \begin{aligned}[t]
        &\min && \lambda +\mu \bar{d} \\
            & st && y \leq d \\
            &&& \mu d_s +\lambda \geq -qy_s - rz_s\\
            &&& y_s+z_s \leq x\\
            &&& y_s, z_s \geq 0\\
            &&& \lambda, \mu \\
     \end{aligned}\\
\end{aligned}
$$

$$
\begin{aligned}
    &\min && cx + \lambda +\mu \bar{d} \\
    & st && 0 \leq x \leq u \\
        &&& y \leq d \\
        &&& \mu d_s +\lambda \geq -qy_s - rz_s\\
        &&& y_s+z_s \leq x\\
        &&& y_s, z_s \geq 0\\
        &&& \lambda, \mu \\
\end{aligned}
$$

In [4]:
struct NewsboyData
    u::Float64
    c::Float64
    q::Float64
    r::Float64
    d::Vector{Float64}
    d_mean::Float64
end

function get_newsboy_data(data::NewsboyData)
    u = data.u
    c = data.c
    q = data.q
    r = data.r
    d = data.d
    d_mean = data.d_mean
    return u, c, q, r, d, d_mean
end

function newsboy_optimal(data::NewsboyData)
    
    u, c, q, r, d, d_mean = get_newsboy_data(data)
    S = length(d)
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    @variable(model, 0<=x<=u)
    @variable(model, 0<=y[s=1:S]<=d[s])
    @variable(model, 0<=z[1:S])
    @variable(model, mu)
    @variable(model, lambda)

    @constraint(model,[s = 1:S], y[s] + z[s] <= x)
    @constraint(model,[s = 1:S], mu*d[s] + lambda ≥ -q * y[s] - r * z[s])

    @objective(model, Min,c*x + lambda + mu*d_mean)

    optimize!(model)
    return objective_value(model), value(model[:x])
end

newsboy_optimal (generic function with 1 method)

In [5]:
data = NewsboyData(150,10,25,5,collect(50:150),100)
obj1, x = newsboy_optimal(data)

(-1250.0, 150.0)

## 2) Formulate a DRO model for the farmer's problem. Assume a choose expected productivity and associated support set to build the ambiguity set. Develop and solve the single level problem.

$$
\begin{aligned}
    &\min && 150a_1 + 230a_2 + 260a_3 + max_{q \geq 0} \Sigma_s q_s Q(a,s)  \\
    & st && a_1+a_2+a_3 = 500\\
        &&& 0 \leq a_1, a_2, a_3 \\
        &&& \Sigma_s q_s = 1\\
        &&& \Sigma_s q_sd_s = \bar{d}\\
\end{aligned}
$$

$$
\begin{aligned}
   Q(a,s) = &\min && 238c_1 + 210c_2 - 170v_1 - 150v_2 - l_3   \\
    & st && p_1 + c_1 - v_1 \geq 200\\
        &&& p_2 + c_2 - v_2 \geq 240\\
        &&& l_3 \leq 36p_3\\
        &&& l_3 \leq 10(p_3-6000) + 36*6000\\
        &&& p_1 = a_1s_1\\
        &&& p_2 = a_2s_2\\
        &&& p_3 = a_3s_3\\
        &&& 0 \leq p_1, p_2, p_3\\
        &&& 0 \leq c_1, c_2, v_1, v_2\\
        &&& 0 \leq l_3
\end{aligned}
$$

$$
\begin{aligned}
    &\min && 150a_1 + 230a_2 + 260a_3 + D(a,s)  \\
    & st && a_1+a_2+a_3 = 500\\
        &&& 0 \leq a_1, a_2, a_3 \\
\end{aligned}
$$

$$
\begin{aligned}
   D(a,s) = &\min && \lambda +\mu \bar{d}   \\
    & st && p_1 + c_1 - v_1 \geq 200\\
        &&& p_2 + c_2 - v_2 \geq 240\\
        &&& l_3 \leq 36p_3\\
        &&& l_3 \leq 10(p_3-6000) + 36*6000\\
        &&& p_1 = a_1s_1\\
        &&& p_2 = a_2s_2\\
        &&& p_3 = a_3s_3\\
        &&& 0 \leq p_1, p_2, p_3\\
        &&& 0 \leq c_1, c_2, v_1, v_2\\
        &&& 0 \leq l_3\\
        &&& \mu s_1 +\lambda \geq 238c_1 + 210c_2 - 170v_1 - 150v_2 - l_3\\
        &&& \mu s_2 +\lambda \geq 238c_1 + 210c_2 - 170v_1 - 150v_2 - l_3\\
        &&& \mu s_3 +\lambda \geq 238c_1 + 210c_2 - 170v_1 - 150v_2 - l_3\\
        &&& \lambda, \mu\\
\end{aligned}
$$

$$
\begin{aligned}
    &\min && 150a_1 + 230a_2 + 260a_3 + \lambda +\mu \bar{d} \\
    & st && a_1+a_2+a_3 = 500\\
        &&& 0 \leq a_1, a_2, a_3 \\
        &&& p_1 + c_1 - v_1 \geq 200\\
        &&& p_2 + c_2 - v_2 \geq 240\\
        &&& l_3 \leq 36p_3\\
        &&& l_3 \leq 10(p_3-6000) + 36*6000\\
        &&& p_1 = a_1s_1\\
        &&& p_2 = a_2s_2\\
        &&& p_3 = a_3s_3\\
        &&& 0 \leq p_1, p_2, p_3\\
        &&& 0 \leq c_1, c_2, v_1, v_2\\
        &&& 0 \leq l_3\\
        &&& \mu s_1 +\lambda \geq 238c_1 + 210c_2 - 170v_1 - 150v_2 - l_3\\
        &&& \mu s_2 +\lambda \geq 238c_1 + 210c_2 - 170v_1 - 150v_2 - l_3\\
        &&& \mu s_3 +\lambda \geq 238c_1 + 210c_2 - 170v_1 - 150v_2 - l_3\\
        &&& \lambda, \mu\\
\end{aligned}
$$

In [14]:
function farmer_optimal()
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    mean = [2.5, 3, 20]
    rate = [2 2.5 3
            2.4 3 3.6
            16 20 24]
    
    @variable(model, 0<=a[1:3])
    @variable(model, 0<=c[1:2,1:3])
    @variable(model, 0<=v[1:2,1:3])
    @variable(model, 0<=l[1:3])
    @variable(model, mu[1:3])
    @variable(model, lambda)
    
    
    @expression(model,p[i=1:3,j=1:3], a[i]*rate[i,j])
    
    
    @constraint(model, sum(a) <= 500)
    @constraint(model, [i=1:3], p[1,i] + c[1,i] - v[1,i] >= 200 )
    @constraint(model, [i=1:3], p[2,i] + c[2,i] - v[2,i] >= 240 )
    @constraint(model, [i=1:3], l[i] <= 36*p[3,i] )
    @constraint(model, [i=1:3], l[i] <= 10*(p[3,i]-6000) + 36*6000)
    
    @constraint(model, [i=1:3], sum(mu.*rate[:,i]) + lambda >=
        238*c[1,i] + 210*c[2,i] - 170*v[1,i] - 150*v[2,i] - l[i])
    
    
    @objective(model, Min, 150a[1] + 230a[2] + 260a[3] + lambda + sum(mu.*mean))

    optimize!(model)
    return objective_value(model), value.(model[:a])
end

farmer_optimal (generic function with 1 method)

In [15]:
farmer_optimal()

(-108250.0, [150.0, 100.0, 250.0])